<a href="https://colab.research.google.com/github/Harshavardhan7299/Keras_Tuner/blob/main/Keras_tunner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Keras Tunner**

In [8]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=596a072b5cfad88809f403cac44fedbcc442efae3e3a287618b81428920d790e
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=97d83bacef81bdf16836c5879b93ea0dd9bb63a693bb33c146618815731ee252
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
train_images = train_images/255.0
test_images = test_images/255.0

In [5]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [6]:
def build_model(hp):
  model = keras.Sequential([
         keras.layers.Conv2D(
             filters = hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
             kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
             activation = 'relu',
             input_shape = (28,28,1)
         ),     
         keras.layers.Conv2D(
             filters = hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
             kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
             activation = 'relu'
         ),    
         keras.layers.Flatten(),
         keras.layers.Dense(
             units = hp.Int('dense_1_units',min_value=32, max_value=128, step =16),
             activation = 'relu'
         ),       
         keras.layers.Dense(10,activation='softmax')])
  
  model.compile(
    optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-1,1e-3])),
    loss= 'sparse_categorical_crossentropy',
    metrics = ['accuracy'])
  
  return model
  

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    directory = 'output',
    project_name = 'MNIST-Fashion'
)

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 11s 6ms/step - loss: 162.1755 - accuracy: 0.1007 - val_loss: 2.3082 - val_accuracy: 0.1055
Epoch 2/3
1688/1688 [==============================] - 11s 6ms/step - loss: 2.3147 - accuracy: 0.1004 - val_loss: 2.3132 - val_accuracy: 0.1032
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 2.3156 - accuracy: 0.0976 - val_loss: 2.3334 - val_accuracy: 0.1008


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 10.3158 - accuracy: 0.0995 - val_loss: 2.3098 - val_accuracy: 0.1003
Epoch 2/3
1688/1688 [==============================] - 10s 6ms/step - loss: 2.3154 - accuracy: 0.0993 - val_loss: 2.3163 - val_accuracy: 0.1032
Epoch 3/3
1688/1688 [==============================] - 10s 6ms/step - loss: 2.3153 - accuracy: 0.0980 - val_loss: 2.3174 - val_accuracy: 0.0942


Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3723 - accuracy: 0.8666 - val_loss: 0.2682 - val_accuracy: 0.9023
Epoch 2/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2241 - accuracy: 0.9177 - val_loss: 0.2425 - val_accuracy: 0.9098
Epoch 3/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.1532 - accuracy: 0.9426 - val_loss: 0.2389 - val_accuracy: 0.9142


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 39.3448 - accuracy: 0.1026 - val_loss: 2.3089 - val_accuracy: 0.1027
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3151 - accuracy: 0.1003 - val_loss: 2.3248 - val_accuracy: 0.0925
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3147 - accuracy: 0.1021 - val_loss: 2.3149 - val_accuracy: 0.1032


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 5.1947 - accuracy: 0.0997 - val_loss: 2.3149 - val_accuracy: 0.0985
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 2.3155 - accuracy: 0.0982 - val_loss: 2.3209 - val_accuracy: 0.0942
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 2.3158 - accuracy: 0.1004 - val_loss: 2.3094 - val_accuracy: 0.0973


INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.fit(train_images,train_labels, epochs=10, validation_data=(test_images,test_labels))

Epoch 1/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.1177 - accuracy: 0.9576 - val_loss: 0.2761 - val_accuracy: 0.9152
Epoch 2/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0699 - accuracy: 0.9742 - val_loss: 0.3170 - val_accuracy: 0.9116
Epoch 3/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0476 - accuracy: 0.9829 - val_loss: 0.3738 - val_accuracy: 0.9131
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0318 - accuracy: 0.9887 - val_loss: 0.4323 - val_accuracy: 0.9123
Epoch 5/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0254 - accuracy: 0.9908 - val_loss: 0.4633 - val_accuracy: 0.9190
Epoch 6/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0215 - accuracy: 0.9924 - val_loss: 0.5074 - val_accuracy: 0.9133
Epoch 7/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0158 - accuracy: 0.9946 - val_loss: 0.5573 - val_ac